<a href="https://colab.research.google.com/github/satvick1234/PRODIGY_ML_05/blob/main/TASK_5_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [ ]:
# Constants
IMAGE_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 101  # Number of food categories in Food-101 dataset

In [ ]:
# Load and preprocess dataset (assumed to be already downloaded and stored)
train_datagen = ImageDataGenerator(
    rescale=1./255,
        shear_range=0.2,
            zoom_range=0.2,
                horizontal_flip=True,
                    validation_split=0.2)  # 20% validation split

In [ ]:
train_generator = train_datagen.flow_from_directory(
      "/content/drive/MyDrive/images",
          target_size=(IMAGE_SIZE, IMAGE_SIZE),
              batch_size=BATCH_SIZE,
                  class_mode='categorical',
                      subset='training')  # set as training data



Found 15138 images belonging to 101 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
      '/content/drive/MyDrive/images',  # same directory as training data
          target_size=(IMAGE_SIZE, IMAGE_SIZE),
              batch_size=BATCH_SIZE,
                  class_mode='categorical',
                      subset='validation')  # set as validation data


Found 3784 images belonging to 101 classes.


In [ ]:
# Load pre-trained EfficientNetB0 model without top layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

In [ ]:
# Freeze base model layers
base_model.trainable = False

In [ ]:
# Add custom top layers for classification and regression
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(512, activation='relu')(x)  # Add more dense layers as needed
classification_output = Dense(NUM_CLASSES, activation='softmax', name='classification')(x)
calorie_output = Dense(1, activation='linear', name='calorie')(x)

In [ ]:
# Combine inputs and outputs into a model
model = Model(inputs=base_model.input, outputs=[classification_output, calorie_output])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss={'classification': 'categorical_crossentropy', 'calorie': 'mean_squared_error'},
                            metrics={'classification': 'accuracy', 'calorie': 'mae'})

In [ ]:
# Train the model
history = model.fit(
    train_generator,
        steps_per_epoch=train_generator.n // BATCH_SIZE,
            epochs=0,
                validation_data=validation_generator,
                    validation_steps=validation_generator.n // BATCH_SIZE)


In [ ]:
# Save the model
model.save('food_classifier_calorie_estimator.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
# Example inference (assuming you have an image and want to predict)
def predict_image(image_path="/content/drive/MyDrive/images/tuna_tartare/3519.jpg"):
    img = tf.keras.preprocessing.image.load_img("/content/drive/MyDrive/images/tuna_tartare/3519.jpg", target_size=(IMAGE_SIZE, IMAGE_SIZE))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    img_array=img_array/ 255.0  # Rescale to [0, 1]

    # Predict using the model
    food_prediction, calorie_prediction = model.predict(img_array)

    # Decode predictions (assuming food_prediction is categorical)
    food_class = np.argmax(food_prediction)

    return food_class, calorie_prediction # Return both predictions

# Example usage
food_class, calorie_prediction = predict_image('3519.jpg')
print(f'Predicted Food Class: {food_class}')
print(f'Estimated Calorie Content: {calorie_prediction}')

1/1 [==============================] - 0s 115ms/step
Predicted Food Class: 99
Estimated Calorie Content: [[0.00381337]]


In [ ]:
# calorie_prediction contains the estimated calorie content

print(food_class, float(calorie_prediction))

99 0.0038133710622787476


<ipython-input-31-ae32b42d10c9>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(food_class, float(calorie_prediction))


In [ ]:
# Example usage
food_class, calorie_prediction = predict_image('3519.jpg')
print(f'Predicted Food Class: {food_class}')
print(f'Estimated Calorie Content: {calorie_prediction}')

1/1 [==============================] - 0s 120ms/step
Predicted Food Class: 99
Estimated Calorie Content: [[0.00381337]]
